In [ ]:
import os 
os.chdir("..")
os.chdir("..")
from src.cpwl.multires.class_multiressolver import *
import matplotlib.pyplot as plt
import torch.nn.functional as func
import torch
from src.cpwl.tools.ptychography import Ptychography2_v2 as Ptychography2
np.random.seed(0)
torch.manual_seed(0)


In [ ]:
def run_test(lmbd,noise_std):
    # Setting the operating device as cpu and the inital as ...
    
    scale = 7
    I_in = 15*np.array([1, 15, 10, 5, 2, 5, 10, 30, 100])
    I_out = 24*np.array([0, 0, 0, 0, 8, 7, 6, 5, 60])
    #I_out = 300*np.array([0, 0, 0, 0, 0, 0, 0, 0, 10])
    cycle = [0, -1, -1, -1, -1,1,  1, 1, 1]
    device = "cuda"
    lmbda = lmbd
    LR = 1e-3
    tol = [1e-10] * 9
    tol_in = [1e-10] * 9
    img_count = 10*10

    linOperator = Ptychography2(in_shape=(2**scale-1, 2**scale-1), n_img=img_count, probe_type='square',
                           probe_radius=32, defocus_factor=0, 
                           fov=170, threshold=0.3, device=device)

    image = plt.imread('samples/peppers_reduced.jpg')[:2**scale-1, :2**scale-1] / 255
    image_tensor = torch.tensor(image).double().to(device).view(1, 1, 2**scale-1, 2**scale-1)
    image_tensor_ = torch.exp(1j * (image_tensor))
    y = linOperator.apply(image_tensor_)
    y = y + torch.abs(noise_std*torch.randn_like(y))
    
    #Initiate the MultiRes class with the inital scale.
    multires = MultiRes(scale, device)
    loss = Loss(linOperator,y, lmbda = lmbda)

    model = MultiResSolver(multires, loss, LR = LR,
                           I_in = I_in,
                           I_out = I_out,
                           tol = tol,
                           tol_in = tol_in,
                           cycle = cycle,
                           l1_type = "l1_row")
    
    model.solve_multigrid()
    model.print_time()
    return model

In [ ]:
noise_std = 10**(-3.5)

modelinf = run_test(lmbd = 1e-16,noise_std=1e-16)
model7 = run_test(lmbd = 1e-7,noise_std=noise_std)
model6 = run_test(lmbd = 1e-6,noise_std=noise_std)
model5 = run_test(lmbd = 1e-5,noise_std=noise_std)
model4 = run_test(lmbd = 1e-4,noise_std=noise_std)
model3 = run_test(lmbd = 1e-3,noise_std=noise_std)

In [ ]:
def unwrap_2d(phase):
    """
    Unwraps a 2D phase array using NumPy's 1D unwrap function.
    
    Parameters:
    phase (numpy array): The 2D phase array to be unwrapped.
    
    Returns:
    unwrapped_phase (numpy array): The 2D unwrapped phase array.
    """
    # Unwrap along the first axis (rows)
    unwrapped_phase = np.unwrap(phase, axis=0)
    
    # Unwrap along the second axis (columns)
    unwrapped_phase = np.unwrap(unwrapped_phase, axis=1)
    
    return unwrapped_phase

In [ ]:
image = plt.imread('samples/peppers_reduced.jpg')[:2**7-1, :2**7-1] / 255
plt.figure(figsize=(15, 18),dpi = 120)
plt.subplot(5, 3, 1)
plt.imshow(image,cmap='gray')
plt.title(r"$(1.a)$ Phase of GT $(\angle x)$ @ $\lambda = 0$")
plt.colorbar()
plt.subplot(5, 3, 2)
phase = torch.angle(modelinf.sols[-1][0,0,:,:].to('cpu'))
phase = unwrap_2d(phase)
phase = (phase - np.min(phase))/(np.max(phase) - np.min(phase))
plt.imshow(phase,cmap = 'gray')
plt.title(r"$(1.b)$ Phase of Reconstruction $(\angle \hat{x})$ @ $\lambda = 0$")
plt.colorbar()
plt.subplot(5, 3, 3)
plt.imshow((np.array(image)-np.array(phase))**2,cmap='gray')
plt.title(r"$(1.c)$ MSE: $(||\angle x-\angle \hat{x}||^2)$ @ $\lambda = 0$") 
plt.colorbar()


plt.subplot(5, 3, 4)
plt.imshow(image,cmap='gray')
plt.title(r"$(2.a)$ Phase of GT $(\angle x)$ @ $\lambda = 10^{-7}$")
plt.colorbar()
plt.subplot(5, 3, 5)
phase = torch.angle(model7.sols[-1][0,0,:,:].to('cpu'))
phase = unwrap_2d(phase)
phase = (phase - np.min(phase))/(np.max(phase) - np.min(phase))
plt.imshow(phase,cmap = 'gray')
plt.title(r"$(2.b)$ Phase of Reconstruction $(\angle \hat{x})$ @ $\lambda = 10^{-7}$")
plt.colorbar()
plt.subplot(5, 3, 6)
plt.imshow(np.abs(np.array(image)-np.array(phase))**2,cmap='gray')
plt.title(r"$(2.c)$ MSE: $(||\angle x-\angle \hat{x}||^2)$ @ $\lambda = 10^{-7}$") 
plt.colorbar()

plt.subplot(5, 3, 7)
plt.imshow(image,cmap='gray')
plt.title(r"$(3.a)$ Phase of GT $(\angle x)$ @ $\lambda = 10^{-6}$")
plt.colorbar()
plt.subplot(5, 3, 8)
phase = torch.angle(model6.sols[-1][0,0,:,:].to('cpu'))
phase = unwrap_2d(phase)
phase = (phase - np.min(phase))/(np.max(phase) - np.min(phase))
plt.imshow(phase,cmap = 'gray')
plt.title(r"$(3.b)$ Phase of Reconstruction $(\angle \hat{x})$ @ $\lambda = 10^{-6}$")
plt.colorbar()
plt.subplot(5, 3, 9)
plt.imshow(np.abs(np.array(image)-np.array(phase))**2,cmap='gray')
plt.title(r"$(3.c)$ MSE: $(||\angle x-\angle \hat{x}||^2)$ @ $\lambda = 10^{-6}$") 
plt.colorbar()

plt.subplot(5, 3, 10)
plt.imshow(image,cmap='gray')
plt.title(r"$(4.a)$ Phase of GT $(\angle x)$ @ $\lambda = 10^{-5}$")
plt.colorbar()
plt.subplot(5, 3, 11)
phase = torch.angle(model5.sols[-1][0,0,:,:].to('cpu'))
phase = unwrap_2d(phase)
phase = (phase - np.min(phase))/(np.max(phase) - np.min(phase))
plt.imshow(phase,cmap = 'gray')
plt.title(r"$(4.b)$ Phase of Reconstruction $(\angle \hat{x})$ @ $\lambda = 10^{-5}$")
plt.colorbar()
plt.subplot(5, 3, 12)
plt.imshow(np.abs(np.array(image)-np.array(phase))**2,cmap='gray')
plt.title(r"$(4.c)$ MSE: $(||\angle x-\angle \hat{x}||^2)$ @ $\lambda = 10^{-5}$") 
plt.colorbar()

plt.subplot(5, 3, 13)
plt.imshow(image,cmap='gray')
plt.title(r"$(5.a)$ Phase of GT $(\angle x)$ @ $\lambda = 10^{-4}$")
plt.colorbar()
plt.subplot(5, 3, 14)
phase = torch.angle(model4.sols[-1][0,0,:,:].to('cpu'))
phase = unwrap_2d(phase)
phase = (phase - np.min(phase))/(np.max(phase) - np.min(phase))
plt.imshow(phase,cmap = 'gray')
plt.title(r"$(5.b)$ Phase of Reconstruction $(\angle \hat{x})$ @ $\lambda = 10^{-4}$")
plt.colorbar()
plt.subplot(5, 3, 15)
plt.imshow(np.abs(np.array(image)-np.array(phase))**2,cmap='gray')
plt.title(r"$(5.c)$ MSE: $(||\angle x-\angle \hat{x}||^2)$ @ $\lambda = 10^{-4}$") 
plt.colorbar()
plt.tight_layout()
